In [28]:
import os
import pandas as pd
import numpy as np 


from Bio import Entrez
from Bio import SeqIO


from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import fcluster

from sklearn.cluster import AgglomerativeClustering


import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [29]:
def find_mash_dist_file(folder):
    folder = os.path.abspath(folder)
    files = os.listdir(folder)
    mash_file = [file for file in files if 'mash.distances.tab' in file][0]
    mash_file_path = os.path.join(folder, mash_file)
    return mash_file_path
    

In [34]:
mash_file = find_mash_dist_file('/home/pjsola/TMP/mashclust_test/')

In [35]:
mash_file

'/home/pjsola/TMP/mashclust_test/K10339_conda_52.coverage_adapted_filtered_80_term.mash.distances.tab'

In [42]:
def mash_dist_to_pairwise(distance_file, distance_type='hash_distance'):
    df = pd.read_csv(distance_file, sep='\t', names=['reference_ID', 'query_ID', 'distance', 'p_value', 'shared_hashes'])
    df[['hash_1', 'hash_2']] = df['shared_hashes'].str.split('/', expand=True)
    df.hash_1 = df.hash_1.astype(float)
    df.hash_2 = df.hash_2.astype(float)
    df['hash_distance'] = 1 - (df.hash_1 / df.hash_2)
    dfpair = df[['reference_ID', 'query_ID', distance_type]]
    
    return dfpair

In [86]:
def pairwise_to_matrix(pwdf):
    dist_matrix = pwdf.groupby(['reference_ID', 'query_ID']).mean().unstack()
    dist_matrix = dist_matrix.droplevel(0, axis=1)
    return dist_matrix

In [46]:
pairwise_distance = mash_dist_to_pairwise(mash_file)

In [47]:
pairwise_distance.head()

,reference_ID,query_ID,hash_distance
0,NZ_CP026852.1,NZ_CP026852.1,0.000
1,NZ_CM004622.1,NZ_CP026852.1,0.996
2,NZ_CP035364.1,NZ_CP026852.1,1.000
3,NZ_CP035347.1,NZ_CP026852.1,1.000
4,NZ_LR025100.1,NZ_CP026852.1,0.997


In [87]:
distamce_matrix = pairwise_to_matrix(pairwise_distance)

In [88]:
distamce_matrix.head()

query_ID,CP029217.1,LR134132.1,NZ_CM004622.1,NZ_CM017034.1,NZ_CM017091.1,NZ_CM017179.1,NZ_CP011334.1,NZ_CP015133.1,NZ_CP015501.1,NZ_CP018443.1,NZ_CP020850.1,NZ_CP023947.1,NZ_CP024516.1,NZ_CP026852.1,NZ_CP027151.1,NZ_CP027154.1,NZ_CP027156.1,NZ_CP027163.1,NZ_CP028818.1,NZ_CP029222.1,NZ_CP031580.1,NZ_CP034282.1,NZ_CP035347.1,NZ_CP035364.1,NZ_CP036325.1,NZ_CP041935.1,NZ_CP042869.1,NZ_CP044032.1,NZ_CP045282.1,NZ_LR025100.1,NZ_LT904874.1,WMHT01000002.1
reference_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CP029217.1,0.000,0.996,1.000,1.0,1.000,1.000,1.000,1.0,0.999,1.000,1.000,0.999,0.999,1.000,1.0,1.0,1.000,1.0,1.000,1.0,1.0,0.999,1.0,1.000,1.0,0.999,1.0,1.000,1.000,1.000,1.0,0.999
LR134132.1,0.996,0.000,1.000,1.0,1.000,1.000,1.000,1.0,1.000,1.000,1.000,1.000,1.000,1.000,1.0,1.0,1.000,1.0,1.000,1.0,1.0,1.000,1.0,1.000,1.0,1.000,1.0,1.000,1.000,1.000,1.0,1.000
NZ_CM004622.1,1.000,1.000,0.000,1.0,0.563,0.476,0.674,1.0,1.000,0.998,0.989,0.973,0.981,0.996,1.0,1.0,0.995,1.0,0.998,1.0,1.0,0.978,1.0,0.999,1.0,1.000,1.0,0.998,0.997,0.998,1.0,1.000
NZ_CM017034.1,1.000,1.000,1.000,0.0,1.000,1.000,0.913,1.0,1.000,1.000,1.000,0.987,0.989,1.000,1.0,1.0,0.997,1.0,1.000,1.0,1.0,0.989,1.0,1.000,1.0,0.994,1.0,1.000,0.999,1.000,1.0,1.000
NZ_CM017091.1,1.000,1.000,0.563,1.0,0.000,0.207,0.882,1.0,1.000,1.000,1.000,0.973,0.981,1.000,1.0,1.0,0.989,1.0,1.000,1.0,1.0,0.978,1.0,1.000,1.0,1.000,1.0,1.000,1.000,1.000,1.0,1.000


In [287]:
pairwise_distance

,reference_ID,query_ID,hash_distance
0,NZ_CP026852.1,NZ_CP026852.1,0.000
1,NZ_CM004622.1,NZ_CP026852.1,0.996
2,NZ_CP035364.1,NZ_CP026852.1,1.000
3,NZ_CP035347.1,NZ_CP026852.1,1.000
4,NZ_LR025100.1,NZ_CP026852.1,0.997
5,NZ_CM017179.1,NZ_CP026852.1,1.000
6,NZ_CM017091.1,NZ_CP026852.1,1.000
7,NZ_CP042869.1,NZ_CP026852.1,0.995
8,NZ_CP045282.1,NZ_CP026852.1,0.984
9,NZ_CP011334.1,NZ_CP026852.1,1.000


In [288]:
pairwise_distance[(pairwise_distance[pairwise_distance.columns[0]] != pairwise_distance[pairwise_distance.columns[1]]) & (pairwise_distance[pairwise_distance.columns[2]] <= 0.5)].sort_values(by=[pairwise_distance.columns[2]])

,reference_ID,query_ID,hash_distance
153,NZ_CP044032.1,NZ_LR025100.1,0.000
804,NZ_LR025100.1,NZ_CP044032.1,0.000
403,NZ_CP028818.1,NZ_CP018443.1,0.079
620,NZ_CP018443.1,NZ_CP028818.1,0.079
1006,NZ_CP024516.1,WMHT01000002.1,0.150
479,WMHT01000002.1,NZ_CP024516.1,0.150
166,NZ_CM017091.1,NZ_CM017179.1,0.207
197,NZ_CM017179.1,NZ_CM017091.1,0.207
140,NZ_CP018443.1,NZ_LR025100.1,0.220
812,NZ_CP018443.1,NZ_CP044032.1,0.220


In [289]:
def pairwise_to_cluster(pw,threshold = 0.5):
    groups = {}
    columns = pw.columns.tolist()
    sorted_df = pw[(pw[columns[0]] != pw[columns[1]]) & (pw[columns[2]] <= threshold)].sort_values(by=[columns[2]])
    
    def rename_dict_clusters(cluster_dict):
        reordered_dict = {}
        for i, k in enumerate(list(cluster_dict)):
            reordered_dict[i] = cluster_dict[k]
        return reordered_dict
    
    def regroup_clusters(list_keys, groups_dict, both_samples_list):
        #sum previous clusters
        list_keys.sort()
        new_cluster = sum([groups_dict[key] for key in list_keys], [])
        #add new cluster
        cluster_asign = list(set(new_cluster + both_samples_list))
        #Remove duped cluster
        first_cluster = list_keys[0]
        groups_dict[first_cluster] = cluster_asign
        rest_cluster = list_keys[1:]
        for key in rest_cluster:
            del groups_dict[key]
        groups_dict = rename_dict_clusters(groups_dict)
        return groups_dict
        
    for _, row in sorted_df.iterrows():
        group_number = len(groups)
        sample_1 = str(row[0])
        sample_2 = str(row[1])
        both_samples_list = row[0:2].tolist()
                
        if group_number == 0:
            groups[group_number] = both_samples_list
        
        all_samples_dict = sum(groups.values(), [])
                
        if sample_1 in all_samples_dict or sample_2 in all_samples_dict:
            #extract cluster which have the new samples
            key_with_sample = {key for (key,value) in groups.items() if (sample_1 in value or sample_2 in value)}
            
            cluster_with_sample = list(key_with_sample)
            cluster_with_sample_name = cluster_with_sample[0]
            number_of_shared_clusters = len(key_with_sample)
            if number_of_shared_clusters > 1:
                groups = regroup_clusters(cluster_with_sample, groups, both_samples_list)
            else:
                groups[cluster_with_sample_name] = list(set(groups[cluster_with_sample_name] + both_samples_list))
        else:
            groups[group_number] = both_samples_list
            
    for _, row in pw[(pw[pw.columns[0]] != pw[pw.columns[1]]) & (pw[pw.columns[2]] > threshold)].iterrows():
        sample_1 = str(row[0])
        sample_2 = str(row[1])
        all_samples_dict = sum(groups.values(), [])
        if sample_1 not in all_samples_dict:
            group_number = len(groups)
            groups[group_number] = [sample_1]
        
        if sample_2 not in all_samples_dict:
            group_number = len(groups)
            groups[group_number] = [sample_2]
            
    cluster_df = pd.DataFrame(groups.values(),index=list(groups))
    
    cluster_df_return = cluster_df.stack().droplevel(1).reset_index().rename(columns={'index': 'group', 0: 'id'})
            
    return cluster_df_return

In [290]:
clusters = pairwise_to_cluster(pairwise_distance,threshold = 0.5)

In [291]:
clusters.head()

,group,id
0,0,NZ_CP028818.1
1,0,NZ_LR025100.1
2,0,NZ_CP018443.1
3,0,NZ_CP044032.1
4,1,NZ_CP023947.1


In [98]:
clusters.groupby('group')['id'].apply(list).reset_index(name='accs')

,group,accs
0,0,"[NZ_CP028818.1, NZ_LR025100.1, NZ_CP018443.1, NZ_CP044032.1]"
1,1,"[NZ_CP023947.1, WMHT01000002.1, NZ_CP024516.1, NZ_CP034282.1]"
2,2,"[NZ_CM017091.1, NZ_CM004622.1, NZ_CM017179.1]"
3,3,"[NZ_CP015501.1, NZ_CP041935.1]"
4,4,[NZ_CP026852.1]
5,5,[NZ_CP035364.1]
6,6,[NZ_CP035347.1]
7,7,[NZ_CP042869.1]
8,8,[NZ_CP045282.1]
9,9,[NZ_CP011334.1]


In [92]:
seqnames = ['NZ_CP023947.1', 'WMHT01000002.1','NZ_CP024516.1', 'NZ_CP034282.1']
distamce_matrix.filter(seqnames, axis=1).filter(seqnames, axis=0)

query_ID,NZ_CP023947.1,WMHT01000002.1,NZ_CP024516.1,NZ_CP034282.1
reference_ID,,,,
NZ_CP023947.1,0.000,0.551,0.550,0.455
WMHT01000002.1,0.551,0.000,0.150,0.393
NZ_CP024516.1,0.550,0.150,0.000,0.333
NZ_CP034282.1,0.455,0.393,0.333,0.000


In [160]:
def calculate_seqlen(fasta_file):
    len_df = pd.DataFrame(columns=['id','length'])
    index = 0
    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        len_df.loc[index, 'id'] = seq_record.id
        len_df.loc[index, 'length'] = len(seq_record)
        index = index + 1
    return len_df
    

In [161]:
fasta_file = '/home/pjsola/Develop/PJSC_utils/data/K10339_conda_52.coverage_adapted_filtered_80_term.fasta'

In [162]:
len_df = calculate_seqlen(fasta_file)

In [163]:
len_df.head()

,id,length
0,NZ_CP026852.1,41246
1,NZ_CM004622.1,3873
2,NZ_CP035364.1,13142
3,NZ_CP035347.1,10695
4,NZ_LR025100.1,63589


In [102]:
"""def acc_to_len_id(row):
    accession_number = row.id
    Entrez.email = "A.N.Other@example.com"
    try:
        handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text")
        record = SeqIO.read(handle, "fasta")
        handle.close()
        #print("Downloaded: " + record.description)
        #print("Downloaded: " + str(len(record)))
        #return record
        description = ' '.join(record.description.split()[3:])
        species = ' '.join(record.description.split()[1:3])
        return len(record), species, description
    except:
        print(record.id + " No record in NCBI, using local data")
        return len(record), 'Unknown', 'Unknown'
        
    #SeqIO.write(record, output_handle, "fasta")
"""

In [103]:
#clusters[['length', 'species', 'description']] = clusters.apply(acc_to_len_id, axis=1, result_type="expand")

In [166]:
final_cluster = clusters.merge(len_df, on='id', how='left')

In [270]:
final_cluster.head()

,group,id,length
0,0,NZ_CP028818.1,46885
1,0,NZ_LR025100.1,63589
2,0,NZ_CP018443.1,50611
3,0,NZ_CP044032.1,63589
4,1,NZ_CP023947.1,153385


In [185]:
final_cluster.sort_values(by=['group', 'length'], ascending=[True, False]).groupby('group').head(1)

,group,id,length
1,0,NZ_LR025100.1,63589
6,1,NZ_CP024516.1,227807
8,2,NZ_CM017091.1,4058
12,3,NZ_CP041935.1,287790
13,4,NZ_CP026852.1,41246
14,5,NZ_CP035364.1,13142
15,6,NZ_CP035347.1,10695
16,7,NZ_CP042869.1,44240
17,8,NZ_CP045282.1,27029
18,9,NZ_CP011334.1,2954


In [260]:
def extract_representative(row):
    row.clustered.remove(row.id)
    return

In [279]:
def retrieve_fasta_cluster(fasta_file, final_cluster, output_dir, quantity_id=1, save_clustered=False):
    input_file = os.path.abspath(fasta_file)
    file_prefix = input_file.split('/')[-1]
    prefix = ('.').join(file_prefix.split('.')[0:-1])
    
    output_representative = os.path.join(output_dir, prefix + '.representative.fasta')
    output_clustered = os.path.join(output_dir, prefix + '.clustered.fasta')
    output_summary = os.path.join(output_dir, prefix + '.clusters.tab')
    
    representative_id = final_cluster.sort_values(by=['group', 'length'], ascending=[True, False]).groupby('group').head(quantity_id)
    summary_id_grouped = final_cluster.groupby('group')['id'].apply(list).reset_index(name='clustered')
    representative_list = representative_id.id.tolist()
    representative_and_sumary = representative_id.merge(summary_id_grouped, on='group', how='left')
    #Use function extract_representative to remove the repr. from column
    representative_and_sumary.apply(extract_representative, axis=1)
    
    #read the fasta and retrieve representative sequences
    representative_records = []
    clustered_records = []
    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        if seq_record.id in representative_list:
            representative_records.append(seq_record)
        else:
            clustered_records.append(seq_record)
        
    SeqIO.write(representative_records, output_representative, "fasta")
    
    if not save_clustered == False:
        SeqIO.write(clustered_records, output_clustered, "fasta")
        
    representative_and_sumary.to_csv(output_summary, sep='\t', index=False)


In [280]:
representative_and_sumary = retrieve_fasta_cluster(fasta_file, final_cluster, '/home/pjsola/TMP/mashclust_test', quantity_id=1, save_clustered=True)